In [ ]:
import pandas as pd

df = pd.read_csv("final_cleaned_dataset.csv")

partner_cols = [c for c in df.columns if "_Export" in c or "_Import" in c]
df["TotalTrade"] = df[partner_cols].sum(axis=1)
df["TDI"] = df[partner_cols].div(df["TotalTrade"].replace(0, 1), axis=0).pow(2).sum(axis=1)

df["GDPpc_norm"] = df["GDP (current US$)"].fillna(0) / df["Total_Population_Both_sexes"].replace(0, 1)

df["ResilienceScore"] = (
    df["GDPpc_norm"].rank(pct=True) +
    (1 - df["TDI"].rank(pct=True)) -
    df["Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)"].rank(pct=True)
)

if "Exports of goods and services (% of GDP)" in df.columns and "Imports of goods and services (% of GDP)" in df.columns:
    df["SpendingEfficiency"] = (
        df["GDP growth (annual %)"].fillna(0) /
        (df["Exports of goods and services (% of GDP)"].fillna(0) +
         df["Imports of goods and services (% of GDP)"].fillna(0)).replace(0, 1)
    )

df["ShockImpact"] = (
    (df["Total Damage, Adjusted ('000 US$)"].fillna(0) + 1) *
    (
        df["TDI"].fillna(0) +
        df["Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)"].fillna(0)
    )
)

df.to_csv("final_with_indexes.csv", index=False)
print("Added indexes (TDI, ResilienceScore, SpendingEfficiency, ShockImpact) to final_with_indexes.csv")


✅ Added indexes (TDI, ResilienceScore, SpendingEfficiency, ShockImpact) to final_with_indexes.csv
